In [12]:
import random
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans


class Agent:
    def __init__(self, org_dim, hid1, hid2, lat_dim, i):
        # Instantiate the VAE with the provided dimensions
        self.vae = VAE(org_dim, hid1, hid2, lat_dim)
        self.num = i
        self.m2s = self.vae.m2s  # encoder part
        self.s2m = self.vae.s2m  # decoder part
        self.m2m = nn.Sequential(self.vae.m2s, self.vae.s2m)  # autoencoder

    def forward(self, x):
        x_hat, z, mean, cov = self.vae.forward(x)
        return mean, cov

    def encode(self, x):
        return self.vae.encode(x)

    def decode(self, z):
        return self.vae.decode(z)


class VAE:
    def __init__(self, org_dim, hid1, hid2, lat_dim):
        self.m2s = nn.Sequential(
            nn.Linear(org_dim, hid1),
            nn.Sigmoid(),
            nn.Linear(hid1, hid2),
            nn.Sigmoid(),
            nn.Linear(hid2, lat_dim),
            nn.Sigmoid()
        )

        self.mean = nn.Linear(lat_dim, 16)
        self.cov = nn.Linear(lat_dim, 16)

        self.s2m = nn.Sequential(
            nn.Linear(lat_dim, hid2),
            nn.Sigmoid(),
            nn.Linear(hid2, hid1),
            nn.Sigmoid(),
            nn.Linear(hid1, org_dim),
            nn.Sigmoid()
        )

    def encode(self, x):
        x = self.m2s(x)
        mean, cov = self.mean(x), self.cov(x)
        return mean, cov

    def decode(self, x):
        return self.s2m(x)

    # def reparameterisation(self, mean, cov):
    #     dist = torch.distributions.Normal(0, cov)
    #     epsilon = dist.sample()
    #     z = mean + cov * epsilon
    #     return z
    def reparameterisation(self, mean, cov):
        std = torch.exp(0.5 * cov)  # Convert covariance to standard deviation
        epsilon = torch.randn_like(std)  # Sample from a normal distribution
        z = mean + std * epsilon
        return z

    def forward(self, x):
        mean, cov = self.encode(x)
        z = self.reparameterisation(mean, cov)
        x_hat = self.s2m(z)
        return x_hat, z, mean, cov


def create_agent(org_dim, hid1, hid2, lat_dim, i):
    return Agent(org_dim, hid1, hid2, lat_dim, i)


def generate_meaning_space(shape, orientation, scale):
    # Load dataset
    dataset_zip = np.load('/Users/hyoyeon/Desktop/UNI/Year 3/Individual Project/Language-Evolution-Modelling-with-ILM/dsprites-dataset-master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz', allow_pickle=True, encoding='latin1')
    imgs = dataset_zip['imgs']
    latents_classes = dataset_zip['latents_classes']
    latents_values = dataset_zip['latents_values']
    selected_indices = np.arange(len(imgs))   # shape (1) square (2) ellipse (3) heart

    if shape is not None:
        selected_indices = selected_indices[latents_classes[selected_indices, 1] == shape]

    if orientation is not None:  # too much, 40 values present.
        selected_indices = selected_indices[latents_values[selected_indices, 3] == orientation]

    if scale is not None:  # scale': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ])
        selected_indices = selected_indices[latents_values[selected_indices, 2] == scale]

    filtered_imgs = imgs[selected_indices]

    # **Flatten images before returning them**
    return [torch.tensor(img.flatten(), dtype=torch.float32) for img in filtered_imgs]



def gen_supervised_data(tutor, all_meanings):
    print("Entered gen supervised data...")
    T = []
    for meaning in all_meanings:
        # print("Shape of meaning before encoding:", meaning.shape)
        signal = tutor.m2s(meaning.unsqueeze(0)).detach().round().squeeze(0)
        T.append((meaning.numpy(), signal.numpy()))
    return T


def gen_unsupervised_data(all_meanings, A_size):
    print("Entered gen unsupervised data...")
    U = []
    for _ in range(A_size):
        meaning = random.choice(all_meanings)
        U.append(meaning.numpy())
    return U


def loss_function(recon_x, x, mean, cov):
    recon_loss = nn.MSELoss()(recon_x, x)
    kl_loss = -0.5 * torch.sum(1 + cov - mean.pow(2) - cov.exp())
    loss = recon_loss + kl_loss
    return loss


In [11]:
pwd

'/Users/hyoyeon/Desktop/UNI/Year 3/Individual Project/Language-Evolution-Modelling-with-ILM/autoencoder'

In [2]:
def train_combined(agent, tutor, A_size, B_size, all_meanings, epochs):
    print("Entered train combined...")
    optimiser_m2s = torch.optim.Adam(agent.m2s.parameters(), lr=5.0)
    optimiser_s2m = torch.optim.Adam(agent.s2m.parameters(), lr=5.0)
    optimiser_m2m = torch.optim.Adam(list(agent.m2s.parameters()) + list(agent.s2m.parameters()),
                                     lr=5.0)  # check if I'm using the optimiser c

    # loss_function = nn.MSELoss()

    T = gen_supervised_data(tutor, all_meanings)
    A = gen_unsupervised_data(all_meanings, A_size)

    m2mtraining = 0

    for epoch in range(epochs):
        print(f"\n===== Epoch {epoch + 1}/{epochs} =====")
        B1 = [random.choice(T) for _ in range(B_size)]
        B2 = B1.copy()
        random.shuffle(B2)

        for i in range(B_size):
            # training encoder
            print(f"Processing {i}th data...")
            optimiser_m2s.zero_grad()
            m2s_meaning, m2s_signal = B1[i]

            m2s_meaning = torch.tensor(m2s_meaning, dtype=torch.float32).unsqueeze(0)
            m2s_signal = torch.tensor(m2s_signal, dtype=torch.float32).unsqueeze(0)
            m2s_mean, m2s_cov = agent.forward(m2s_meaning)
            pred_m2s = agent.m2s(m2s_meaning)

            loss_m2s = loss_function(pred_m2s, m2s_signal, m2s_mean, m2s_cov)
            loss_m2s.backward()
            optimiser_m2s.step()

            # training decoder
            optimiser_s2m.zero_grad()
            s2m_meaning, s2m_signal = B2[i]
            s2m_signal = torch.tensor(s2m_signal, dtype=torch.float32).unsqueeze(0)
            s2m_meaning = torch.tensor(s2m_meaning, dtype=torch.float32).unsqueeze(0)

            pred_s2m = agent.s2m(s2m_signal)
            loss_s2m = nn.MSELoss()(pred_s2m, s2m_meaning)
            loss_s2m.backward()
            optimiser_s2m.step()

            # unsupervised training
            meanings_u = [random.choice(A) for _ in range(20)]
            for meaning in meanings_u:
                optimiser_m2m.zero_grad()
                auto_m = torch.tensor(meaning, dtype=torch.float32).unsqueeze(0)

                pred_m2m = agent.m2m(auto_m)
                loss_auto = nn.MSELoss()(pred_m2m, auto_m)
                loss_auto.backward()
                optimiser_m2m.step()

                m2mtraining += 1


In [3]:
def iterated_learning(x_dim, h_dim1, h_dim2, lat_dim, all_meanings, generations=20, A_size=75, B_size=75, epochs=20):
    print("Entered iterated learning")
    tutor = create_agent(x_dim, h_dim1, h_dim2, lat_dim, 1)

    stability_scores = []
    expressivity_scores = []
    compositionality_scores = []

    for gen in range(1, generations + 1):
        pupil = create_agent(x_dim, h_dim1, h_dim2, lat_dim, gen)
        train_combined(pupil, tutor, A_size, B_size, all_meanings, epochs)

        stability_scores.append(stability(tutor, pupil, all_meanings))
        expressivity_scores.append(expressivity(pupil, all_meanings))
        compositionality_scores.append(compositionality(pupil, all_meanings))

        tutor = pupil
        # print(f"Tutor gen: {tutor.num}")

    return stability_scores, expressivity_scores, compositionality_scores
    # return None


In [5]:

def plot_results(stability_scores, expressivity_scores, compositionality_scores, generations, replicates=25):
    plt.figure(figsize=(15, 5))
    gens = np.arange(1, generations + 1)

    # Define colors
    colors = {'stability': 'purple', 'expressivity': 'blue', 'compositionality': 'orange',
              's': (0.5, 0.0, 0.5, 0.1),  # Light purple (RGBA with low alpha)
              'x': (0.0, 0.0, 1.0, 0.1),  # Light blue
              'c': (1.0, 0.65, 0.0, 0.1)  # Light orange
              }

    # Stability Plot
    plt.figure(figsize=(6, 4))
    for rep in stability_scores:
        plt.plot(gens, rep, color=colors['s'], alpha=0.2)
    plt.plot(gens, np.mean(stability_scores, axis=0), color=colors['stability'], linewidth=3)
    plt.xlabel("Generations", fontsize=12)
    plt.ylabel("s", fontsize=12)
    # plt.title("Stability Over Generations", fontsize=14)
    plt.show()

    # Expressivity Plot
    plt.figure(figsize=(6, 4))
    for rep in expressivity_scores:
        plt.plot(gens, rep, color=colors['x'], alpha=0.2)
    plt.plot(gens, np.mean(expressivity_scores, axis=0), color=colors['expressivity'], linewidth=3)
    plt.xlabel("Generations", fontsize=12)
    plt.ylabel("x", fontsize=12)
    # plt.title("Expressivity Over Generations", fontsize=14)
    plt.show()

    # Compositionality Plot
    plt.figure(figsize=(6, 4))
    for rep in compositionality_scores:
        plt.plot(gens, rep, color=colors['c'], alpha=0.2)
    plt.plot(gens, np.mean(compositionality_scores, axis=0), color=colors['compositionality'], linewidth=3)
    plt.xlabel("Generations", fontsize=12)
    plt.ylabel("c", fontsize=12)
    # plt.title("Compositionality Over Generations", fontsize=14)
    plt.show()


def stability(tutor, pupil, all_meanings):
    tutor.m2s.eval()
    pupil.s2m.eval()

    with torch.no_grad():
        tutor_latents = []
        pupil_latents = []

        for meaning in all_meanings:
            m = meaning.clone().detach().float().unsqueeze(0)

            # Corrected: First encode, then decode
            encoded_m = pupil.m2s(m)  # Encode to latent space (1, 16)
            pupil_latents.append(pupil.s2m(encoded_m).squeeze(0).numpy())  # Decode properly

            tutor_encoded_m = tutor.m2s(m)  # Encode to latent space (1, 16)
            tutor_latents.append(tutor_encoded_m.squeeze(0).numpy())  # Store latent representation

    # Convert to numpy arrays
    tutor_latents = np.array(tutor_latents)
    pupil_latents = np.array(pupil_latents)

    # Run t-SNE on both generations
    tsne = TSNE(n_components=2, random_state=42)
    tutor_2d = tsne.fit_transform(tutor_latents)
    pupil_2d = tsne.fit_transform(pupil_latents)

    # Compute stability score based on similarity of representations
    similarity = np.mean([1 - cosine(t, p) for t, p in zip(tutor_2d, pupil_2d)])
    print(f"{pupil.i} stability: {similarity}")
    return similarity  # Higher = more stable



def expressivity(agent, all_meanings):
    agent.m2s.eval()

    with torch.no_grad():
        latents = []
        for meaning in all_meanings:
            latents.append(agent.m2s(meaning.unsqueeze(0)).squeeze(0).numpy())

    # Convert to numpy
    latents = np.array(latents)

    # Run t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    latent_2d = tsne.fit_transform(latents)

    # Compute dispersion (average pairwise Euclidean distance)
    distances = np.sqrt(np.sum((latent_2d[:, None, :] - latent_2d[None, :, :]) ** 2, axis=-1))
    expressivity_score = np.mean(distances)

    return expressivity_score  # Higher = more diverse meanings


def calculate_entropy(p):
    if p == 0 or p == 1:
        return 0
    else:
        return -p * np.log2(p) - (1 - p) * np.log2(1 - p)


def compositionality(agent, all_meanings, n_clusters=5):
    agent.m2s.eval()

    with torch.no_grad():
        latents = []
        for meaning in all_meanings:
            latents.append(agent.m2s(meaning.unsqueeze(0)).squeeze(0).numpy())

    # Convert to numpy
    latents = np.array(latents)

    # Run t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    latent_2d = tsne.fit_transform(latents)

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(latent_2d)

    # Compute cluster separation (higher separation = more compositional)
    cluster_separation = np.mean(
        [np.linalg.norm(latent_2d[i] - latent_2d[j]) for i in range(len(latent_2d)) for j in range(len(latent_2d)) if
         labels[i] == labels[j]])

    return cluster_separation  # Higher = more compositional


In [ ]:

all_meanings = generate_meaning_space(shape=1, orientation=None, scale=0.5)
generations = 50
replicates = 25

stability_scores = []
expressivity_scores = []
compositionality_scores = []

# for i in range(replicates):
#     print(f"============================================{i}============================================")
stability, expressivity, compositionality = iterated_learning(4096, 2048, 512, 16, all_meanings)
stability_scores.append(stability)
# expressivity_scores.append(expressivity)
# compositionality_scores.append(compositionality)
#


Entered iterated learning
Entered train combined...
Entered gen supervised data...
Entered gen unsupervised data...

===== Epoch 1/20 =====
Processing 0th data...
Processing 1th data...
Processing 2th data...
Processing 3th data...
Processing 4th data...
Processing 5th data...
Processing 6th data...
Processing 7th data...
Processing 8th data...
Processing 9th data...
Processing 10th data...
Processing 11th data...
Processing 12th data...
Processing 13th data...
Processing 14th data...
Processing 15th data...
Processing 16th data...
Processing 17th data...
Processing 18th data...
Processing 19th data...
Processing 20th data...
Processing 21th data...
Processing 22th data...
Processing 23th data...
Processing 24th data...
Processing 25th data...
Processing 26th data...
Processing 27th data...
Processing 28th data...
Processing 29th data...
Processing 30th data...
Processing 31th data...
Processing 32th data...
Processing 33th data...
Processing 34th data...
Processing 35th data...
Proces

Processing 34th data...
Processing 35th data...
Processing 36th data...
Processing 37th data...
Processing 38th data...
Processing 39th data...
Processing 40th data...
Processing 41th data...
Processing 42th data...
Processing 43th data...
Processing 44th data...
Processing 45th data...
Processing 46th data...
Processing 47th data...
Processing 48th data...
Processing 49th data...
Processing 50th data...
Processing 51th data...
Processing 52th data...
Processing 53th data...
Processing 54th data...
Processing 55th data...
Processing 56th data...
Processing 57th data...
Processing 58th data...
Processing 59th data...
Processing 60th data...
Processing 61th data...
Processing 62th data...
Processing 63th data...
Processing 64th data...
Processing 65th data...
Processing 66th data...
Processing 67th data...
Processing 68th data...
Processing 69th data...
Processing 70th data...
Processing 71th data...
Processing 72th data...
Processing 73th data...
Processing 74th data...

===== Epoch 6/2

Processing 74th data...

===== Epoch 10/20 =====
Processing 0th data...
Processing 1th data...
Processing 2th data...
Processing 3th data...
Processing 4th data...
Processing 5th data...
Processing 6th data...
Processing 7th data...
Processing 8th data...
Processing 9th data...
Processing 10th data...
Processing 11th data...
Processing 12th data...
Processing 13th data...
Processing 14th data...
Processing 15th data...
Processing 16th data...
Processing 17th data...
Processing 18th data...
Processing 19th data...
Processing 20th data...
Processing 21th data...
Processing 22th data...
Processing 23th data...
Processing 24th data...
Processing 25th data...
Processing 26th data...
Processing 27th data...
Processing 28th data...
Processing 29th data...
Processing 30th data...
Processing 31th data...
Processing 32th data...
Processing 33th data...
Processing 34th data...
Processing 35th data...
Processing 36th data...
Processing 37th data...
Processing 38th data...
Processing 39th data...
P

Processing 38th data...
Processing 39th data...
Processing 40th data...
Processing 41th data...
Processing 42th data...
Processing 43th data...
Processing 44th data...
Processing 45th data...
Processing 46th data...
Processing 47th data...
Processing 48th data...
Processing 49th data...
Processing 50th data...
Processing 51th data...
Processing 52th data...
Processing 53th data...
Processing 54th data...
Processing 55th data...
Processing 56th data...
Processing 57th data...
Processing 58th data...
Processing 59th data...
Processing 60th data...
Processing 61th data...
Processing 62th data...
Processing 63th data...
Processing 64th data...
Processing 65th data...
Processing 66th data...
Processing 67th data...
Processing 68th data...
Processing 69th data...
Processing 70th data...
Processing 71th data...
Processing 72th data...
Processing 73th data...
Processing 74th data...

===== Epoch 15/20 =====
Processing 0th data...
Processing 1th data...
Processing 2th data...
Processing 3th dat

Processing 1th data...
Processing 2th data...
Processing 3th data...
Processing 4th data...
Processing 5th data...
Processing 6th data...
Processing 7th data...
Processing 8th data...
Processing 9th data...
Processing 10th data...
Processing 11th data...
Processing 12th data...
Processing 13th data...
Processing 14th data...
Processing 15th data...
Processing 16th data...
Processing 17th data...
Processing 18th data...
Processing 19th data...
Processing 20th data...
Processing 21th data...
Processing 22th data...
Processing 23th data...
Processing 24th data...
Processing 25th data...
Processing 26th data...
Processing 27th data...
Processing 28th data...
Processing 29th data...
Processing 30th data...
Processing 31th data...
Processing 32th data...
Processing 33th data...
Processing 34th data...
Processing 35th data...
Processing 36th data...
Processing 37th data...
Processing 38th data...
Processing 39th data...
Processing 40th data...
Processing 41th data...
Processing 42th data...
P

In [ ]:
plot_results(stability_scores, expressivity_scores, compositionality_scores, generations, replicates)
